# Project 4 - Shopify redesigned with Firebase

In [1]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

In [2]:
batch = db.batch()

apps = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_cat = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
cat = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
ben = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
ppf = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
plans = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
rev = pd.read_csv('shopify/reviews.csv', sep=',', header=0, lineterminator='\n')

row_apps = apps.values.tolist()
row_ac = apps_cat.values.tolist()
row_cat = cat.values.tolist()
row_ben = ben.values.tolist()
row_ppf = ppf.values.tolist()
row_plans = plans.values.tolist()
row_rev = rev.values.tolist()

## Populating collections

### Populating the categories collection

In [3]:
print("Records found in categories:", len(row_cat))

# Field names for reviews categories per ERD:
# cat_id, title, app_id
for row in row_cat:
    
    record = {}
    record['cat_id'] = row[0]
    record['title'] = row[1]
    record['app_id'] = []
    
    for rowCheck in row_ac:
        if row[0] == rowCheck[1]:
            record['app_id'].append(rowCheck[0])
    
    class_ref = db.collection('categories').document(row[0])
    
    batch.set(class_ref, record)
    
silent = batch.commit()
print("Total added records:", len(silent))

Records found in categories: 12
Total added records: 12


### Populating the reviews collection

In [4]:
k = 0
print("Records found in reviews:", len(row_rev))
checks = []
added = 0
batch = db.batch()

# Field names for reviews collection per ERD:
# review_id, app_id, author, rating, posted_at
for row in row_rev:
    record = {}
    record['review_id'] = row[0] + '_' + str(row[1]).replace("/","_fw_slash_") #replace any / with fw_slash
    
    checks.append(record['review_id'])

    record['app_id'] = row[0]
    record['author'] = row[1]
    record['rating'] = row[2]
    record['posted_at'] = row[3]

    class_ref = db.collection('reviews').document(record['review_id'])

    batch.set(class_ref, record)
    k += 1

    if k > 100:
        added += k
        batch.commit()
        k = 0
    
silent = batch.commit()
print("Records added:", added + len(silent))

Records found in reviews: 124601
Records added: 124601


### Populating the pricing_plans collection

In [5]:
batch = db.batch()

k = 0

# Field names for pricing_plans collection per ERD:
# plan_id, app_id, title, price, features
print("Records found in pricing_plans:", len(row_plans))
for row in row_plans:
    k += 1
    
    record = {}
    record['plan_id'] = row[0]
    record['app_id'] = row[1]
    record['title'] = row[2]
    record['price'] = row[3]
    record['features'] = []
    
    for rowCheck in row_ppf:
        if row[0] == rowCheck[1]:
            record['features'].append(rowCheck[2])
    
    class_ref = db.collection('pricing_plans').document(row[0])
    
    batch.set(class_ref, record)
    
    if k > 100:
        batch.commit()
        k = 0

batch.commit()
print("Records added:", len(list(db.collection('pricing_plans').list_documents())))

Records found in pricing_plans: 6275
Records added: 6275


### Populating the apps collection

In [6]:
batch = db.batch()

k = 0
added = 0
print("Records found in apps:", len(apps))

# Field names for apps collection per ERD:
# app_id, url, title, developer, developer_link, icon, rating, reviews_count, 
# plan_id, key_benefits, review_id, cat_id
for row in row_apps:
    k += 1
    
    record = {}
    record['app_id'] = row[0]
    record['url'] = row[1]
    record['title'] = row[2]
    record['developer'] = row[3]
    record['developer_link'] = row[4]
    record['icon'] = row[5]
    record['rating'] = row[6]
    record['reviews_count'] = row[7]
    
    record['plan_id'] = []
    record['key_benefits'] = []
    record['cat_id'] = []
    record['review_id'] = []
    
    #populate cat_id
    for rowCheck in row_ac:
        if row[0] == rowCheck[0]:
            record['cat_id'].append(rowCheck[1])
    
    #populate cat_id
    for rowCheck in row_plans:
        if row[0] == rowCheck[1]:
            record['plan_id'].append(rowCheck[0])
            
    #populate review_id
    for rowCheck in row_rev:
        if row[0] == rowCheck[0]:
            record['review_id'].append(rowCheck[0] + '_' + str(rowCheck[1]).replace("/","_fw_slash_"))
    
    #populate key_benefits embedded table
    for rowCheck in row_ben:
        if row[0] == rowCheck[0]:
            mapTest = {}
            mapTest['title'] = rowCheck[1]
            mapTest['description'] = rowCheck[2]
            record['key_benefits'].append(mapTest)
        
    
    class_ref = db.collection('apps').document(row[0])
    
    batch.set(class_ref, record)
    
    if k > 100:
        added += k
        batch.commit()
        k = 0

print("Records added:", added + len(batch.commit()))

Records found in apps: 3547
Records added: 3547


## Document counts for each collection

In [7]:
print("Number of documents in the categories collection:", len(list(db.collection('categories').list_documents())))
print("Number of documents in the pricing_plans collection:", len(list(db.collection('pricing_plans').list_documents())))
print("Number of documents in the apps collection:", len(list(db.collection('apps').list_documents())))
print("Number of documents in the reviews collection:", len(list(db.collection('reviews').list_documents())))

Number of documents in the categories collection: 12
Number of documents in the pricing_plans collection: 6275
Number of documents in the apps collection: 3547
Number of documents in the reviews collection: 124601


## Queries of Interest

1) List the 10 apps with the highest number of reviews (based on
apps.review_count). Return the id, title, developer, rating and
reviews_count of those apps. Order the results by reviews_count in
descending order.

In [8]:
a_ref = db.collection('apps')
query = a_ref.select(('app_id', 'title', 'developer', 'rating', 'reviews_count')).order_by('reviews_count', direction='DESCENDING').limit(10)
results = list(query.stream())

for result in results:
    temp = result.to_dict()
    print(f'{result.id} => {temp}')

print("Total results:", len(results))

d9f142ee-b141-4dc4-9353-173db61d2eb0 => {'app_id': 'd9f142ee-b141-4dc4-9353-173db61d2eb0', 'rating': 4.7, 'developer': 'Privy', 'title': 'Privy ‑ Exit Pop Ups & Email', 'reviews_count': 23078}
78ea0810-c008-4a4e-a82f-de0c790e3286 => {'rating': 4.9, 'app_id': '78ea0810-c008-4a4e-a82f-de0c790e3286', 'developer': 'Hextom', 'title': 'Free Shipping Bar', 'reviews_count': 8737}
b88488b0-9912-44d3-b736-224c36f09d95 => {'app_id': 'b88488b0-9912-44d3-b736-224c36f09d95', 'rating': 4.8, 'developer': 'CartKit', 'title': 'Sales Pop ‑ Popup Notification', 'reviews_count': 6905}
e528a60e-94f8-4e92-80e2-5bc6013b8283 => {'app_id': 'e528a60e-94f8-4e92-80e2-5bc6013b8283', 'rating': 4.8, 'developer': 'Grizzly Apps', 'title': 'BEST Currency Converter', 'reviews_count': 5986}
be2640c4-01b5-4d52-9f68-cae8c0734d0d => {'reviews_count': 5596, 'rating': 4.8, 'app_id': 'be2640c4-01b5-4d52-9f68-cae8c0734d0d', 'title': 'Recart FB Messenger Marketing', 'developer': 'Recart'}
70bff9e0-4316-4cc6-84ce-92fcd1bc6925 => {

2) Find the apps which are categorized as “Trust and security”. Return the id, url,
title, developer and rating of those apps. Order the results by rating in
descending order. Limit the number of results to 5.

In [9]:
cat_ref = db.collection('categories')
query = cat_ref.where('title', '==', 'Trust and security')
results = query.stream()

for result in results:
    temp = result.to_dict()
    
#print(temp['cat_id'])
app_id_list = temp['app_id']
#print(len(app_id_list))

app_ref = db.collection('apps')
query = app_ref.select(('app_id', 'url', 'title', 'developer', 'rating')).order_by('rating', direction='DESCENDING')
results = query.stream()

selected = []
for result in results:
    temp = result.to_dict()
    if temp['app_id'] in app_id_list:
        selected.append(temp)
        print(f'{result.id} => {temp}')
    if len(selected) >= 5:
        break

print("Total results:", len(selected))

fe2d6047-9654-494d-9eed-1f00e1468639 => {'title': 'OC Check', 'developer': 'Ocean info', 'url': 'https://apps.shopify.com/age-checker-1', 'app_id': 'fe2d6047-9654-494d-9eed-1f00e1468639', 'rating': 5.0}
f5454a98-8bdb-499d-906a-bf8d9992b6e8 => {'url': 'https://apps.shopify.com/right-click-defender', 'rating': 5.0, 'app_id': 'f5454a98-8bdb-499d-906a-bf8d9992b6e8', 'developer': 'Adolab', 'title': 'Right Click Defender'}
ee95fc91-86c0-41b6-9d04-10422e0c42e8 => {'url': 'https://apps.shopify.com/eu-cookie-privacy-solution', 'app_id': 'ee95fc91-86c0-41b6-9d04-10422e0c42e8', 'rating': 5.0, 'title': 'EU Cookie Consent Banner', 'developer': 'webserviceXXL'}
e525d067-418e-49cb-b0c8-9081c85fe5bd => {'developer': 'Webkul Software Pvt Ltd', 'title': 'Social Commerce', 'url': 'https://apps.shopify.com/social-e-commerce', 'app_id': 'e525d067-418e-49cb-b0c8-9081c85fe5bd', 'rating': 5.0}
e45d25ac-2939-4b88-9052-c6c033ff25d9 => {'rating': 5.0, 'app_id': 'e45d25ac-2939-4b88-9052-c6c033ff25d9', 'title': 'G

3) Find all the apps that are under the Premium Plan and whose price is $12.99 or higher. Return the id, title, developer, rating, and reviews_count
of those apps. Order the results by price in ascending order. Limit the number of results to 10.

In [11]:
#get all app_id in pricing plan where title is Premium Plan and price is higher than 12.99 and order by price
plan_ref = db.collection('pricing_plans')
query = plan_ref.where('price', '>=', 12.99).where('title', '==', 'Premium Plan').order_by('price', direction='ASCENDING')
results = query.stream()

apps = []
k = 0
for result in results:
    k += 1
    temp = result.to_dict()
    apps.append(temp['app_id'])

#print(k)
#print(len(apps))
#get all apps
app_ref = db.collection('apps')
query = app_ref.select(('app_id','title','developer', 'rating', 'reviews_count'))
results = list(query.stream())

selected = []

k = 0
for a in apps:
    for result in results:
        temp = result.to_dict()
        if temp['app_id'] == a:
            k += 1
            selected.append(temp)
            print(f'{result.id} => {temp}')
    if k >= 10:
        break

print(len(selected), "records returned")

a8171589-48cb-4a3a-83f1-5da7a20f5c03 => {'reviews_count': 95, 'rating': 4.9, 'app_id': 'a8171589-48cb-4a3a-83f1-5da7a20f5c03', 'title': 'Enorm Image Slider', 'developer': 'enormapps'}
18a74341-10cf-4a49-af8b-505ec047592d => {'app_id': '18a74341-10cf-4a49-af8b-505ec047592d', 'rating': 5.0, 'title': 'Email Alchemy', 'developer': 'Oiizes', 'reviews_count': 1}
ae85f088-0627-4368-8c2e-16be4cac506d => {'app_id': 'ae85f088-0627-4368-8c2e-16be4cac506d', 'rating': 5.0, 'developer': 'Zooomy', 'title': 'Wishlist ‑ Wishify', 'reviews_count': 161}
7b570802-fd42-49bf-a9f2-19d69838c049 => {'rating': 4.7, 'app_id': '7b570802-fd42-49bf-a9f2-19d69838c049', 'developer': 'Hextom', 'title': 'Countdown Timer Bar', 'reviews_count': 527}
b1441cac-3052-4f43-aacc-9312d1d6ec6b => {'app_id': 'b1441cac-3052-4f43-aacc-9312d1d6ec6b', 'rating': 0.0, 'developer': 'Application Nexus', 'title': 'Magic Ball ‑ Email Popup', 'reviews_count': 0}
3918da5e-152a-4a00-ad27-4fdfba27c9e5 => {'rating': 4.8, 'app_id': '3918da5e-152